#### Tacotron2 + WaveGlow 한국어 TTS 최적화를 위한 필수 전처리 과정을 단계별로 정리한 내용
1. 텍스트 정제, 자소분리, vocab 생성 및 tokenize
2. 음성 데이터 전처리
3. 멜스펙트로그램 변환

In [ ]:
!pip install librosa numpy noisereduce jamo webrtcvad matplotlib sox sentencepiece


### 1) 텍스트 전처리
- 정제
- 자소분리 (음소 단위 변환)
- vocab 생성

##### 텍스트 정제

In [ ]:
import re

def normalize_numbers(text):
    """숫자를 한글 발음으로 변환"""
    num_dict = {
        "0": "영", "1": "일", "2": "이", "3": "삼", "4": "사",
        "5": "오", "6": "육", "7": "칠", "8": "팔", "9": "구"
    }
    return "".join(num_dict.get(c, c) for c in text)

def clean_text(text):
    """텍스트 정제"""
    text = text.lower()  # 소문자로 변환
    text = re.sub(r'[^가-힣0-9\s,.!?]', '', text)  # 한글, 숫자, 공백, 구두점만 유지
    text = re.sub(r'\s+', ' ', text).strip()  # 연속된 공백 제거
    text = normalize_numbers(text)  # 숫자 변환
    return text

# 테스트
sample_text = "100cm짜리 2024년 모델입니다!"
print(clean_text(sample_text))  # "일공공 센티미터짜리 이공이사년 모델입니다!"


##### 자소 분리

In [ ]:
import jamo

def split_jaso(text):
    """한글을 자소 단위로 분리"""
    return " ".join(jamo.hangul_to_jamo(text))

# 테스트
print(split_jaso("안녕하세요"))  # "ㅇㅏㄴ ㄴㅕㅇ ㅎㅏ ㅅㅔ ㅇㅛ"

##### Tokenizer 및 Vocab 생성

In [ ]:
import sentencepiece as spm

# 학습용 코퍼스 저장
corpus_texts = ["안녕하세요", "반갑습니다", "이 모델은 TTS 시스템입니다."]
with open("corpus.txt", "w", encoding="utf-8") as f:
    for line in corpus_texts:
        f.write(line + "\n")

# SentencePiece 모델 학습
spm.SentencePieceTrainer.train(input="corpus.txt", model_prefix="tokenizer", vocab_size=500, model_type="bpe")

# Tokenizer 불러오기
sp = spm.SentencePieceProcessor()
sp.load("tokenizer.model")

# 테스트
print(sp.encode_as_pieces("이 모델은 TTS 시스템입니다."))  # 예: ['▁이', '▁모델은', '▁TTS', '▁시스템입니다']

### 2) 음성 데이터 전처리
- 샘플링 레이트 변환
- 무음 제거
- 볼륨 정규화
- 잡음 제거
- 데이터 길이 통일

##### 샘플링 레이트 변환 (16kHz -> 22.05kHz)

In [ ]:
import librosa

# 음성 데이터 로드
y, sr = librosa.load("audio.wav", sr=16000)

# 22050Hz로 변환
y_resampled = librosa.resample(y, orig_sr=sr, target_sr=22050)

# 저장
librosa.output.write_wav("resampled_audio.wav", y_resampled, 22050)

##### 무음 제거

In [ ]:
import librosa.effects

y, sr = librosa.load("resampled_audio.wav", sr=22050)

# 무음 제거
y_trimmed, _ = librosa.effects.trim(y, top_db=20)

librosa.output.write_wav("trimmed_audio.wav", y_trimmed, sr)

##### 볼륨 정규화

In [ ]:
import numpy as np

def normalize_audio(y):
    return y / np.max(np.abs(y))  # 최대 진폭을 1로 정규화

y, sr = librosa.load("trimmed_audio.wav", sr=22050)
y_norm = normalize_audio(y)

librosa.output.write_wav("normalized_audio.wav", y_norm, sr)


##### 잡음 제거

In [ ]:
import noisereduce as nr

y, sr = librosa.load("normalized_audio.wav", sr=22050)

# 잡음 제거
y_denoised = nr.reduce_noise(y=y, sr=sr)

librosa.output.write_wav("denoised_audio.wav", y_denoised, sr)

### 3) 멜 스펙트로그램 생성 및 시각화
- 멜 스펙트로그램 변환
- 멜 스펙트로그램 시각화

##### 멜 스펙트로그램 변환

In [ ]:
import librosa.display
import matplotlib.pyplot as plt

y, sr = librosa.load("denoised_audio.wav", sr=22050)

# 멜스펙트로그램 변환
mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=80, hop_length=256, win_length=1024, fmax=8000)

# 로그 변환
log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)

##### 멜 스펙트로그램 시각화

In [ ]:
plt.figure(figsize=(10, 4))
librosa.display.specshow(log_mel_spec, sr=sr, x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
plt.show()

### 🔍 정리: 전체 전처리 프로세스

##### ✅ 텍스트 전처리
- 텍스트 정제 (clean_text())
- 자소 분리 (split_jaso())
- 토크나이저 (sentencepiece 활용)

##### ✅ 음성 데이터 전처리
- 샘플링 레이트 통일 (16kHz → 22.05kHz)
- 무음 제거 (librosa.effects.trim())
- 볼륨 정규화
- 잡음 제거 (noisereduce 활용)

##### ✅ 멜스펙트로그램 변환 및 시각화
- 80개 Mel 필터 사용
- Hop length 256, Win length 1024
- fmax=8000 설정
- 로그 스펙트로그램 변환 후 시각화